In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import _pickle as cPickle
import gzip
import os
import sys
import timeit
from sklearn.metrics import confusion_matrix
import math
import os
import pickle

In [3]:
# Import Keras with TensorFlow backend
# Calculate the time it takes to import Keras with Theano backend
start_time = timeit.default_timer()
import keras
end_time = timeit.default_timer()
print("It took Keras with TensorFlow as backend %.1fs to load." % (end_time - start_time))

Using TensorFlow backend.


It took Keras with TensorFlow as backend 0.7s to load.


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

from keras_tqdm import TQDMNotebookCallback
from keras_tqdm import TQDMCallback
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot

from keras import optimizers

In [5]:
# Import MNIST dataset
from keras.datasets import mnist

In [6]:
# shuffle and split the data between training and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
# set the no of training and test dataset examples
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# create placeholders
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


# no of training and test batches
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 784)
60000 train samples
10000 test samples


In [8]:
n_output = len(np.unique(y_train))
print("No of output classes: ", n_output)

No of output classes:  10


In [9]:
# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, num_classes)
y_train = keras.utils.to_categorical(y_train, n_output)
print(y_train.shape)

# Y_test = np_utils.to_categorical(y_test, num_classes)
y_test = keras.utils.to_categorical(y_test, n_output)
print(y_test.shape)

(60000, 10)
(10000, 10)


## Create the CNN Model

In [10]:
# Create a MLP models with two hidden layers, with each hidden layer consisting of 
model = Sequential()

model.add(Dense(10, activation='sigmoid', input_dim=x_train.shape[1]))

## Print the Model Architecture

In [11]:
# model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850.0
Trainable params: 7,850
Non-trainable params: 0.0
_________________________________________________________________


In [12]:
# model visualization - using keras utility functions
img = plot_model(model, to_file='mnist_log_reg.png')

## Optimize the Model

In [13]:
# optimize the model for categorical classification

sgd = optimizers.SGD(lr=0.01, clipnorm=1.)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

## Save Model Checkpoints

In [14]:
# create model checkpoints
save_dir = "checkpoints_mnist_log_reg_keras_tensorflow/"
# create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
# path for the checkpoint file
save_path = os.path.join(save_dir, "mnist_log_reg.hdf5")
if os.path.exists(save_path):
    print("Checkpoint created.")
else:
    print("No checkpoint exists as of now!!!!")

checkpointer = ModelCheckpoint(filepath=save_path, verbose=1, save_best_only=True)

No checkpoint exists as of now!!!!


## Helper Class to Record Loss History

In [15]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

## Helper Function to Initiate Training

In [16]:
# Define a function for training the model

def optimize(batch_size, n_epochs):
    
    print("Starting Optimization")
    
    start_time = timeit.default_timer()
    
    history = LossHistory()
    
    model.fit(x_train, y_train, batch_size=batch_size, verbose=1, epochs=n_epochs, 
              callbacks=[TQDMCallback(), checkpointer, history], 
              validation_data=(x_test, y_test))
    
    end_time = timeit.default_timer()
    
    time_diff = end_time - start_time
    
    print("The code ran for %.1fs" % (time_diff))
    
    print("The code ran for %.2fm" % (time_diff / 60.))

In [17]:
# Peform Actual Training

optimize(batch_size=256, n_epochs=100)

Starting Optimization


Epoch: 0:   0%|          | 0/60000 [00:00<?, ?it/s]

Train on 60000 samples, validate on 10000 samples
Epoch 1/100



Epoch: 0 - loss: 2.296, acc: 0.137  0%|          | 256/60000 [00:00<02:17, 434.17it/s]

 4864/60000 [=>............................] - ETA: 7s - loss: 2.2643 - acc: 0.1565  

...

53504/60000 [=========================>....] - ETA: 0s - loss: 0.3168 - acc: 0.9132


Epoch: 99 - loss: 0.316, acc: 0.913 94%|█████████▍| 56576/60000 [00:00<00:00, 93942.06it/s]

58880/60000 [============================>.] - ETA: 0s - loss: 0.3158 - acc: 0.9137


Epoch: 99 - loss: 0.315, acc: 0.914, val_loss: 0.304, val_acc: 0.918100%|██████████| 60000/60000 [00:00<00:00, 66392.88it/s]
Training: 100%|██████████| 100/100 [01:17<00:00,  1.42it/s]

Epoch 00099: val_loss improved from 0.30412 to 0.30376, saving model to checkpoints_mnist_log_reg_keras_tensorflow/mnist_log_reg.hdf5
60000/60000 [==============================] - 0s - loss: 0.3153 - acc: 0.9137 - val_loss: 0.3038 - val_acc: 0.9177
The code ran for 77.5s
The code ran for 1.29m


In [18]:
# evaluate the model
# find the test score and accuracy
score = model.evaluate(x_test, y_test, verbose=0)

print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.303761698341
Test accuracy: 0.9177
